# Imputation avec Sklearn

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import SGDClassifier

## 1. SimpleImputer

Le simple, fait une application en fonction de la stratégie. Ici, on calcul la moyenne des deux colonnes pour remplacer les nan par cette valeure.
On à entrainer cet valeurs sur X, cela sera conserver pour le X test.

Toujours séparer les imputers en test et train

In [2]:
from sklearn.impute import SimpleImputer

In [3]:
X = np.array([[10, 3],
              [0, 4],
              [5, 3],
             [np.nan, 3]])

imputer = SimpleImputer(missing_values=np.nan,
             strategy='mean')

imputer.fit_transform(X)

array([[10.,  3.],
       [ 0.,  4.],
       [ 5.,  3.],
       [ 5.,  3.]])

In [4]:
X_test = np.array([[12, 5],
                   [40, 2],
                   [5, 5],
                   [np.nan, np.nan]])

imputer.transform(X_test)

array([[12.  ,  5.  ],
       [40.  ,  2.  ],
       [ 5.  ,  5.  ],
       [ 5.  ,  3.25]])

## 2. KNNImputer

In [5]:
from sklearn.impute import KNNImputer

In [6]:
X = np.array([[1, 100],
             [2, 30],
             [3, 15],
             [np.nan, 20]])

imputer = KNNImputer(n_neighbors=1)
imputer.fit_transform(X)

array([[  1., 100.],
       [  2.,  30.],
       [  3.,  15.],
       [  3.,  20.]])

In [7]:
X_test = np.array([[np.nan, 35]])

imputer.transform(X_test)

array([[ 2., 35.]])

## 3. MissingIndicator

In [8]:
from sklearn.impute import MissingIndicator
from sklearn.pipeline import make_union

In [9]:
X = np.array([[1, 100],
             [2, 30],
             [3, 15],
             [np.nan, np.nan]])

MissingIndicator().fit_transform(X)

array([[False, False],
       [False, False],
       [False, False],
       [ True,  True]])

In [10]:
pipeline = make_union(SimpleImputer(strategy='constant', fill_value=-99),
                     MissingIndicator())

pipeline.fit_transform(X)

array([[  1., 100.,   0.,   0.],
       [  2.,  30.,   0.,   0.],
       [  3.,  15.,   0.,   0.],
       [-99., -99.,   1.,   1.]])

Utile pour chercher un type d'échentillon, par exemple, un membre d'équipage qui se serrait perdu dans la base de donnée des voyageurs. Avec make Union, on peut faire simultanément appliqué deux transformers.

Technique d'un datascientiste est d'exploiter le manque d'information pour voir si ça peut donner une info supplémentaire à notre model.

## 4. Application

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split

In [12]:
titanic = sns.load_dataset('titanic')
X = titanic[['pclass', 'age']]
y = titanic['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [13]:
model = make_pipeline(KNNImputer(), SGDClassifier())

params = {'knnimputer__n_neighbors' : [1, 2, 3, 4]}

grid = GridSearchCV(model, param_grid=params, cv=5)

grid.fit(X_train, y_train)
grid.best_params_

{'knnimputer__n_neighbors': 4}